In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch import optim
from tqdm import tqdm
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device is", device)

device is cpu


In [112]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [155]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.kaiming_uniform_(m.weight)
        
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.layer_0 = nn.Conv2d(3, 8, 3, stride=2, padding=1)
        self.layer_1 = nn.Conv2d(8, 16, 3, stride=2, padding=1)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Conv2d(16, 32, 3, stride=2, padding=1)
        self.layer_3 = nn.Conv2d(32, 64, 4)
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(3, 8, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3),
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, padding=1),#, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 8, 3, padding=1),#, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 3, 3, padding=1),#, output_padding=1),
            nn.Tanh()
        )


    def forward(self, x):
#         x = self.layer_0(x)
#         x = self.layer_1(x)
#         x = self.relu(x)
#         x = self.layer_2(x)
#         print(x.shape)
#         x = self.layer_3(x)
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
        

In [156]:
model = Autoencoder()
model.apply(weights_init)

Autoencoder(
  (layer_0): Conv2d(3, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (layer_1): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu): ReLU()
  (layer_2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (layer_3): Conv2d(32, 64, kernel_size=(4, 4), stride=(1, 1))
  (encoder): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv

In [157]:
def train_model(train_model, batch_train_loader, optimizer, loss_fn):
    train_model.train()
    losses_within_batch = []
#     f1_scores_within_batch = []
    for i, data in tqdm(enumerate(batch_train_loader), total=len(batch_train_loader), leave=False):
        
        optimizer.zero_grad()
        input_data = data[0].to(device)
#         print(input_data.shape)
        pred = model(input_data)
        tmp_loss = loss_fn(pred, data[0])
#         import pdb; pdb.set_trace()
        losses_within_batch.append(tmp_loss.item())
        tmp_loss.backward()
        optimizer.step()
    final_loss = np.mean(losses_within_batch)
    return final_loss, train_model

In [116]:
single_data = []
for i, data in enumerate(trainloader):
    single_data.append(data[0])
    break
trainloader = torch.utils.data.DataLoader(single_data)

In [158]:
criterion = nn.MSELoss()
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
all_losses = []
for epoch in range(10000):
    loss, model = train_model(model, trainloader, optimizer, criterion)
    print("{} {}".format(epoch, loss))
    all_losses.append(loss)

  0%|          | 0/1 [00:00<?, ?it/s]        

0 0.9809399247169495


  0%|          | 0/1 [00:00<?, ?it/s]        

1 0.6272134780883789


  0%|          | 0/1 [00:00<?, ?it/s]        

...

  0%|          | 0/1 [00:00<?, ?it/s]        

5580 0.0005511196795850992


  0%|          | 0/1 [00:00<?, ?it/s]        

5581 0.0005495406803674996


KeyboardInterrupt: 

In [7]:
all_losses

[0.042064087714473175,
 0.018709902033109696,
 0.01554885087862506,
 0.014075666315667689,
 0.01333336699372652,
 0.012869144249433367,
 0.012537014010461835,
 0.0122976815381629,
 0.01211461486677405,
 0.011961927136938997,
 0.011864285654509327,
 0.01175086944997406,
 0.0116956333927119,
 0.011615321473340651,
 0.011586109347288721,
 0.011536240103992413,
 0.011498241071897349,
 0.011455901131577318,
 0.011425243036956148,
 0.011396323249664012]

In [59]:
for i, data in enumerate(trainloader):
    print(model.encoder(data[0]))
    break

tensor([[[[ 1.1559e+00,  1.4057e+00],
          [ 9.9431e-01,  1.1936e+00]],

         [[-5.8601e-01,  6.8735e-01],
          [ 2.1972e-01,  4.3159e-01]],

         [[ 1.8502e+00, -1.1120e+00],
          [ 1.4705e+00,  2.3194e+00]],

         [[ 8.5924e-01, -1.8409e+00],
          [-3.3910e+00, -2.4985e+00]],

         [[ 5.2317e-01, -4.8518e-01],
          [-8.2308e-01,  2.1409e-01]],

         [[-7.2073e-01,  6.8150e-01],
          [ 3.9846e-02, -6.0744e-01]],

         [[ 4.1565e-01,  7.9093e-01],
          [-2.3246e-01, -7.0610e-02]],

         [[ 1.7236e-01,  7.3137e-01],
          [ 2.6435e-03, -4.8407e-01]],

         [[ 3.7025e-01,  1.2162e+00],
          [-2.3700e-02, -1.2112e-01]],

         [[ 1.9329e+00,  1.7050e+00],
          [ 1.0369e+00,  1.6050e+00]],

         [[ 2.7713e-01,  1.3263e+00],
          [-4.8476e-01, -1.5196e+00]],

         [[ 8.2897e-01,  6.5164e-01],
          [ 6.6835e-01,  6.0357e-01]],

         [[ 8.8046e-01,  1.1469e+00],
          [ 4.4636e-01,  9

In [60]:
data[0]

tensor([[[[0.4745, 0.4431, 0.4510,  ..., 0.5843, 0.5843, 0.5765],
          [0.4431, 0.4118, 0.4275,  ..., 0.5529, 0.5451, 0.5451],
          [0.4745, 0.4510, 0.4431,  ..., 0.5137, 0.5059, 0.4980],
          ...,
          [0.5216, 0.5216, 0.5294,  ..., 0.5294, 0.5529, 0.5373],
          [0.6078, 0.6000, 0.6078,  ..., 0.6314, 0.6314, 0.6000],
          [0.6471, 0.6314, 0.6314,  ..., 0.5843, 0.6000, 0.5608]],

         [[0.3804, 0.3569, 0.3647,  ..., 0.4980, 0.4980, 0.4902],
          [0.3569, 0.3255, 0.3412,  ..., 0.4667, 0.4588, 0.4588],
          [0.3882, 0.3647, 0.3569,  ..., 0.4275, 0.4196, 0.4118],
          ...,
          [0.4275, 0.4196, 0.4275,  ..., 0.4275, 0.4510, 0.4353],
          [0.5137, 0.5137, 0.5216,  ..., 0.5451, 0.5451, 0.5137],
          [0.5608, 0.5451, 0.5451,  ..., 0.4980, 0.5137, 0.4745]],

         [[0.3490, 0.3255, 0.3333,  ..., 0.4667, 0.4667, 0.4588],
          [0.3176, 0.2941, 0.3098,  ..., 0.4353, 0.4275, 0.4275],
          [0.3569, 0.3333, 0.3255,  ..., 0

In [61]:
model(data[0])

tensor([[[[0.4726, 0.4430, 0.4512,  ..., 0.5826, 0.5846, 0.5751],
          [0.4444, 0.4146, 0.4313,  ..., 0.5517, 0.5498, 0.5453],
          [0.4737, 0.4539, 0.4416,  ..., 0.5136, 0.5008, 0.4985],
          ...,
          [0.5230, 0.5212, 0.5236,  ..., 0.5262, 0.5559, 0.5345],
          [0.6064, 0.6014, 0.6074,  ..., 0.6302, 0.6337, 0.5943],
          [0.6439, 0.6240, 0.6281,  ..., 0.5860, 0.6061, 0.5612]],

         [[0.3842, 0.3557, 0.3629,  ..., 0.4984, 0.4974, 0.4939],
          [0.3540, 0.3250, 0.3401,  ..., 0.4686, 0.4572, 0.4592],
          [0.3877, 0.3656, 0.3578,  ..., 0.4304, 0.4215, 0.4165],
          ...,
          [0.4229, 0.4229, 0.4300,  ..., 0.4288, 0.4500, 0.4409],
          [0.5184, 0.5166, 0.5275,  ..., 0.5441, 0.5440, 0.5149],
          [0.5624, 0.5438, 0.5501,  ..., 0.4968, 0.5119, 0.4755]],

         [[0.3482, 0.3250, 0.3352,  ..., 0.4679, 0.4675, 0.4571],
          [0.3205, 0.2937, 0.3086,  ..., 0.4358, 0.4255, 0.4268],
          [0.3575, 0.3310, 0.3243,  ..., 0

In [132]:
model.encoder(data[0]).shape

torch.Size([32, 64, 1, 1])

In [63]:
data[0].shape

torch.Size([1, 3, 32, 32])